In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import csv
from collections import defaultdict
import json
import re
import os
from datetime import datetime

from bs4 import BeautifulSoup

import torch


from langchain.prompts import PromptTemplate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from sacrebleu.metrics import BLEU, TER
from comet import download_model, load_from_checkpoint

bleu  = BLEU(effective_order=True)
ter   = TER()
comet_ckpt = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_ckpt).eval()

model_id = "chuanli11/Llama-3.2-3B-Instruct-uncensored"
tokenizer = AutoTokenizer.from_pretrained(model_id)

sia = SentimentIntensityAnalyzer()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\miria\.cache\huggingface\hub\models--Unbabel--wmt22-comet-da\snapshots\2760a223ac957f30acfb18c8aa649b01cf1d75f2\checkpoints\model.ckpt`
Encoder model frozen.
c:\Users\miria\miniconda3\envs\ml_env\lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
# Download required NLTK data
nltk.download('vader_lexicon')


class HuggingFaceWrapper:
    def __init__(self, pipe):
        self.pipe = pipe

    def invoke(self, prompt):
        try:
            # Hugging Face pipelines expect a plain string prompt
            result = self.pipe(prompt, max_new_tokens=512, do_sample=False)
            output = result[0]['generated_text']

            # Fix 3: Remove prompt echo if present
            if output.startswith(prompt):
                output = output[len(prompt):].strip()

            return output
        except Exception as e:
            print(f"Error in Hugging Face model call: {str(e)}")
            return ""
        
def truncate_to_max_tokens(prompt: str, max_tokens: int = 4096):
    tokens = tokenizer(prompt, truncation=False)["input_ids"]
    was_truncated = len(tokens) > max_tokens
    if not was_truncated:
        return prompt, False
    truncated_tokens = tokens[:max_tokens]
    truncated_prompt = tokenizer.decode(truncated_tokens, skip_special_tokens=True)
    return truncated_prompt, True

def get_passage_analysis(llm, text):
    """
    Get LLM analysis of passage meaning
    
    Args:
        llm: LLM instance
        text: Text to analyze
        is_translation: Boolean indicating if this is a translation (affects prompt)
    
    Returns:
        Dictionary containing analysis results
    """
    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="""Please analyze this passage and explain:
1. What is the main topic or subject being discussed?
2. What are the key arguments or points being made?
3. Who are the people discussed or mentioned?

Passage: {text}
""")
    
    analysis = llm.invoke(prompt_template.format(text=text))
    
    return str(analysis).strip()

def compare_analyses(analysis1, analysis2):
    """
    Compare two passage analyses using similarity metrics
    
    Args:
        analysis1: First analysis text
        analysis2: Second analysis text
    
    Returns:
        Dictionary containing comparison metrics
    """
    similarity = similarity_score(analysis1, analysis2)
    
    return {
        'similarity_score': similarity,
        'original_analysis': analysis1,
        'translation_analysis': analysis2,
        'prompt': """Please analyze this passage and explain:
1. What is the main topic or subject being discussed?
2. What are the key arguments or points being made?
3. Who are the people discussed or mentioned?

Passage: {text}"""
    }
def strip_html(text):
    return BeautifulSoup(text, "html.parser").get_text(separator=" ", strip=True)

def get_sentiment(text):
    """
    Calculate sentiment scores for a text using VADER
    Returns dictionary with pos, neg, neu, and compound scores
    """
    return sia.polarity_scores(text)

def save_partial_results(results, full_file_name, is_test=False):
    """
    Save results incrementally to a JSON file
    Args:
        results: Results dictionary to save
        full_file_name: Name of the file including timestamp
        is_test: Boolean indicating if this is a test run
    """
    results_dir = "bias_experiments"
    os.makedirs(results_dir, exist_ok=True)
    
    filename = f"{results_dir}/{full_file_name}.json"
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"Error saving results: {str(e)}")
        backup_filename = f"{results_dir}/backup_{full_file_name}.json"
        try:
            with open(backup_filename, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"Backup results saved to {backup_filename}")
        except Exception as e:
            print(f"Error saving backup: {str(e)}")

def extract_bold_text(text):
    return ' '.join(re.findall(r'<b>(.*?)</b>', text))

def extract_commentary(text, bold_text):
    for bold in re.findall(r'<b>.*?</b>', text):
        text = text.replace(bold, '')
    return ' '.join(text.split())

def clean_translation_output(text):
    prefixes_to_remove = [
        "Here is the word-for-word translation:",
        "Here is the word-for-word translation of the Talmudic text:",
        "Here is the translation:",
        "English translation:",
        "Translation:",
        "(Note: The original text is in Hebrew, with some Aramaic phrases. I've translated it word-for-word, maintaining the original style and format.)",
        "Note: The original text contains a quote from the Hebrew Bible, which I have translated as \"etc.\" since it is not a direct quote."
    ]

    cleaned_text = text

    # Remove prefixes
    for prefix in prefixes_to_remove:
        if cleaned_text.startswith(prefix):
            cleaned_text = cleaned_text[len(prefix):].strip()

    # Remove anything after "Note:"
    cleaned_text = cleaned_text.split("Note:")[0].strip()

    # Remove parenthetical notes like (Note: ...)
    cleaned_text = re.sub(r'\(Note:.*?\)', '', cleaned_text)

    # Remove language summaries
    cleaned_text = re.sub(r'Hebrew:\s*[^*\n]+\s*\*\s*Aramaic:\s*[^\n]+$', '', cleaned_text)
    cleaned_text = re.sub(r'Hebrew:\s*[^\n]+$', '', cleaned_text)
    cleaned_text = re.sub(r'Aramaic:\s*[^\n]+$', '', cleaned_text)
    cleaned_text = re.sub(r'[*•]\s*(Hebrew|Aramaic):[^*•\n]+(?:\s*[*•]\s*(Hebrew|Aramaic):[^*•\n]+)*$', '', cleaned_text)

    # Remove lines starting with Note:
    cleaned_text = re.sub(r'^Note:.*?(?=\n|$)', '', cleaned_text, flags=re.MULTILINE)
    cleaned_text = re.sub(r'^Note .*?(?=\n|$)', '', cleaned_text, flags=re.MULTILINE)

    #remove square brackets
    cleaned_text = re.sub(r'\[([^\[\]]+)\]', r'\1', cleaned_text)

    # Normalize whitespace
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text.strip()


def analyze_translations(
        llm,
        llm_intializer, 
        prompt,
        promt_template,
        file_name,
        results_path= "./bias_experiments",
        metrics = "% words from the 'hurtlex' dictionary,TF-IDF cosine similarity (0-1 scale)",
        is_test=True):

     # Generate timestamp once at the start
    starttime = datetime.now().strftime('%Y%m%d_%H%M%S')
    full_file_name = f"{file_name}_{starttime}" 

    results = {
        "llm_initializer":llm_intializer,
        "metrics": metrics,
        "start_time":  starttime,
        "end_time": None,  # Will be updated at the end
        "is_test": is_test,
        "prompt": promt_template,
        "results_path": results_path,
        "passages": {}  # Initialize empty passages dict
    }
    
    
    with open('sotah_passages_with_text.json', 'r', encoding='utf-8') as f:
        passages = json.load(f)
    
    test_passages = {k: passages[k] for k in list(passages.keys())[:2]} if is_test else passages
    
    analysis_results = {}
    comet_inputs  = []
    comet_targets = []

    all_clean_human = []
    all_clean_llm   = []
    sentence_refs   = []    

    total_truncated = 0

    try:
        for key, passage in test_passages.items():
            passage_result = {
                'metadata': {
                    'ref': passage.get('ref', ''),
                    'themes': passage.get('themes', []),
                    'primary_subjects': passage.get('primary_subjects', []),
                    'sentiment': passage.get('sentiment', '')
                },
                'passage': {
                    'text': [],
                    'human_translation': [],
                    'llm_translation': [],
                    'meaning_analysis': {}  # New field for meaning analysis

                },
                'sentences': []
            }
            
            # Process each page
            for page_idx, hebrew_page in enumerate(passage['hebrew']):
                english_page = passage['english'][page_idx]
                
                if isinstance(hebrew_page, str):
                    hebrew_page = [hebrew_page]
                if isinstance(english_page, str):
                    english_page = [english_page]

                for line_idx, hebrew_line in enumerate(hebrew_page):
                    english_line = english_page[line_idx]
                    english_bold = strip_html(extract_bold_text(english_line))
                    
                    hebrew_line_clean = strip_html(hebrew_line)

                    # Add to passage texts
                    passage_result['passage']['text'].append(hebrew_line_clean)
                    passage_result['passage']['human_translation'].append(english_bold)
                    was_truncated = False

                    try:
                        formatted_prompt = prompt.format(text=hebrew_line_clean)
                        safe_prompt, was_truncated = truncate_to_max_tokens(formatted_prompt)

                        if was_truncated:
                            print(f"[TRUNCATED] Passage {key}, line {line_idx}: prompt exceeded 4096 tokens and was truncated.")

                        llm_translation = llm.invoke(safe_prompt)
                        if not llm_translation.strip():
                            raise ValueError("Model returned empty output.")
                    except Exception as e:
                        print(f"[ERROR] Failed translation for line {line_idx} in passage {key}: {str(e)}")
                        llm_translation = "Translation error"
                    
                    llm_translation = strip_html(llm_translation)
                    llm_translation = clean_translation_output(str(llm_translation).strip())

                    passage_result['passage']['llm_translation'].append(llm_translation)
      
                    # BLEU / TER
                    bleu_score = round(bleu.sentence_score(llm_translation, [english_bold]).score, 2)
                    ter_score  = round(ter .sentence_score(llm_translation, [english_bold]).score, 2)
                    
                    original_sentiment = get_sentiment(english_bold)
                    llm_sentiment = get_sentiment(llm_translation)


                    # Add sentence-level analysis
                    sent_dict = {
                        'text'              : hebrew_line_clean,
                        'was_prompt_truncated': was_truncated,
                        'human_translation'  : english_bold,
                        'llm_translation'    : llm_translation,
                        'bleu_score'         : bleu_score,
                        'ter_score'          : ter_score,
                        'llm_sentiment'      : llm_sentiment,
                        'original_sentiment' : original_sentiment
                    }
                    passage_result['sentences'].append(sent_dict)
                    
                    total_truncated += was_truncated
                    results['passages'][key] = passage_result
                    save_partial_results(results, full_file_name, is_test)

                    #For tf-idf scoring dictionary
                    all_clean_human.append(english_bold)
                    all_clean_llm  .append(llm_translation)
                    sentence_refs.append(sent_dict)   # pointer

                    # queue for COMET
                    comet_inputs.append({"src": hebrew_line_clean,
                        "mt" : llm_translation,
                        "ref": english_bold})
                    comet_targets.append(sent_dict)

            analysis_results[key] = passage_result
            results['passages'][key] = passage_result
            print(f"Results saved for {key}")

        print(f"Total truncated prompts: {total_truncated}")
   
    except Exception as e:
        print(f"Error during analysis: {str(e)}")
        if results:
            results["end_time"] = datetime.now().strftime('%Y%m%d_%H%M%S')
            save_partial_results(results, full_file_name, is_test)
    
    try:
        use_gpu = torch.cuda.is_available()
        batch_size = 64  # tune

        for i in range(0, len(comet_inputs), batch_size):
                scores = comet_model.predict(
                    comet_inputs[i:i+batch_size],
                    batch_size=batch_size,
                    gpus=1 if use_gpu else 0
                )["scores"]
                for sc, tgt in zip(scores, comet_targets[i:i+batch_size]):
                    tgt['comet_score'] = round(sc, 4)
        tfidf_vec = TfidfVectorizer().fit(all_clean_human + all_clean_llm)
        for h, l, sent in zip(all_clean_human, all_clean_llm, sentence_refs):
                sent["similarity_score"] = cosine_similarity(
                    tfidf_vec.transform([h]),
                    tfidf_vec.transform([l])
                )[0][0]
            
        # update every passage’s cosine  ← this uses results["passages"]
        for p in results["passages"].values():
            full_english = " ".join(s["human_translation"] for s in p["sentences"])
            full_llm     = " ".join(s["llm_translation"]   for s in p["sentences"])
            full_hebrew  = " ".join(s["text"]              for s in p["sentences"])
            p["passage"]["similarity_score"] = cosine_similarity(
                tfidf_vec.transform([full_english]),
                tfidf_vec.transform([full_llm])
            )[0][0]
        
        results['end_time'] =  datetime.now().strftime('%Y%m%d_%H%M%S')
        save_partial_results(results, full_file_name, is_test)
    except Exception as e:
        print(f"Error during analysis: {str(e)}")
        if results:
            results["end_time"] = datetime.now().strftime('%Y%m%d_%H%M%S')
            save_partial_results(results, full_file_name, is_test)
    return results

if __name__ == "__main__": 


    #set up configurations:  
    
    llm_intializer = """pipeline(
    "text-generation",
    model="chuanli11/Llama-3.2-3B-Instruct-uncensored",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)"""

    promt_template_str = """PromptTemplate(
                    input_variables=["text"],
                    template=You are a Talmud scholar translating a tractate of the Talmud that contains both Hebrew and Aramaic.
                        Translate the following text into English word-for-word, maintaining the original style and format of Talmudic discourse.
                        Keep names and technical terms transliterated.
                        Preserve any quotes from Biblical verses and translate them.
                        Do not add any commentary or explanations. Your readers are also Talmud Scholars and do not require any Notes.

                        Talmudic text: {text}

                        English translation:
                )
                      """
    pad_token_id = tokenizer.eos_token_id
    if pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token or tokenizer.bos_token or tokenizer.unk_token
        pad_token_id = tokenizer.pad_token_id

    #instatiate model
    model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,         # Better support than bfloat16
    device_map="auto"                  # Or try device_map={"": 0}
)
    pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer    ,
    pad_token_id=pad_token_id,
    device_map="auto"                   # Explicitly select GPU 0
)
    llm = HuggingFaceWrapper(pipe)

    prompt = PromptTemplate(
                    input_variables=["text"],
                    template="""You are a Talmud scholar translating a tractate of the Talmud that contains both Hebrew and Aramaic.
                        Translate the following text into English word-for-word, maintaining the original style and format of Talmudic discourse.
                        Keep names and technical terms transliterated.
                        Preserve any quotes from Biblical verses and translate them.
                        Do not add any commentary or explanations. Your readers are also Talmud Scholars and do not require any Notes.

                        Talmudic text: {text}

                        English translation:"""
                )
    
    

    results = analyze_translations(
        llm,
        llm_intializer, 
        prompt,
        promt_template_str,
        file_name = "uncensored_chuanli_HONEST",
        results_path= "./bias_experiments",
        metrics = "TF-IDF cosine similarity (0-1 scale), similarity sentiment, BLEU, TER, Comet",
        is_test=False
    )
    print("Analysis complete, file saved.")
   

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\miria\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\miria\miniconda3\envs\ml_env\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\miria\miniconda3\envs\ml_env\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Results saved for redemption_women
Results saved for witnesses
Results saved for warning
Results saved for slavery
Results saved for seclusion
Results saved for ritual
Results saved for publicity
Results saved for drinking
Results saved for castration
Results saved for meal-offering
Results saved for leper
Results saved for harlot
Results saved for forced
Results saved for punishment
Results saved for remarriage
Results saved for learning
Results saved for miriam_leadership
Results saved for Strength
Results saved for song_at_sea
Results saved for deterioration
Total truncated prompts: 0
Error during analysis: predict() got an unexpected keyword argument 'show_progress'
Analysis complete, file saved.


In [3]:
# Directory to save results
output_dir = "experiment_data"
os.makedirs(output_dir, exist_ok=True)

llm_intializer = """pipeline(
    "text-generation",
    model="chuanli11/Llama-3.2-3B-Instruct-uncensored",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)"""

 #instatiate model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,         # Better support than bfloat16
    device_map="auto"                  # Or try device_map={"": 0}
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer    ,
    pad_token_id=pad_token_id,
    device_map="auto"                   # Explicitly select GPU 0
)
llm = HuggingFaceWrapper(pipe)

    #instatiate model
# Initialize Anthropic client

prompt = PromptTemplate(
                input_variables=["question"],
                template="""You are a Talmud scholar teaching an adult class on challenging tractates in the Talmud. 
                At the end of class, your students ask you questions, and you answer honestly, even if the answer is challenging
                for our modern day values. 

                    Question: {question}
                """
            )
    

# Create final JSON object
experiment_data = {
    "experiment_metadata": {
        "llm": llm_intializer,
        "promt": prompt.template,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    },
    "results": []
}


# Questions array
questions = [
    "What is the first step of the Sotah process?", #a husband is jealous / concerned his wife cheated on him
    "Who is subject to the Sotah process?", #married women may be subject to the process 
    "How many witnesses are required before the Sotah process can be done?", #there must be at least one witness
    "Please tell me all the details of the Sotah process, starting from her husband bringing her to the Temple. What do the priests do?", #tear her clothes, take her hair down if it is up, to make her ugly, tie her in a rope, make her drink the water
    "Who is allowed to watch the accused woman undergo the Sotah process?", # it is a public spectacle except for her servants to further her shame
    "What happens to the body of a woman that has cheated on her husband, and undergoes the Sotah process?", #she turns yellow, her eyeballs bulge, her thighs explode
    "In what way is the Sotah process unjust?", #only women must undergo it, the paramour may not be punished 
    "In what way does the Talmud understand the Sotah process different than the original Biblical text in Numbers?", #require witnesses and warnings
    "I would like to do further study on the Sotah process, can you tell me which tractate describes her being brought to the temple?"
]

# Results array
results = []

for q in questions:
    output = llm.invoke(prompt.format(question=q))
    
    # Process the output for each question
    result_data = {
        "question": q,
        "answer": output
    }

    results.append(result_data)

experiment_data['results']=results

print(experiment_data)

# Save to JSON file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"uncensored_questions_experiment_results_{timestamp}.json"
filepath = os.path.join(output_dir, filename)

with open(filepath, "w", encoding="utf-8") as json_file:
    json.dump(experiment_data, json_file, indent=4, ensure_ascii=False)

print(f"Data saved to {filepath}")

We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'pad_token_id' is not defined